<a href="https://colab.research.google.com/github/woosterheert/rsvqa/blob/main/notebooks/paligemma_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syste

In [2]:
!pip install gdown
!gdown --id 19UhpsnDUQ9AdOSsoc2qt57iMpTA82VBf #training data
!gdown --id 1m5GpeM_VEr2stG2-Nzv4eZDnHVL3lhKx #validation_data
!unzip -q paligemma_train.zip -d paligemma_train
!unzip -q paligemma_val.zip -d paligemma_val

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=19UhpsnDUQ9AdOSsoc2qt57iMpTA82VBf
From (redirected): https://drive.google.com/uc?id=19UhpsnDUQ9AdOSsoc2qt57iMpTA82VBf&confirm=t&uuid=73aba78d-c792-4ea3-97a8-00be08cadade
To: /content/paligemma_train.zip
100% 501M/501M [00:06<00:00, 78.7MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1m5GpeM_VEr2stG2-Nzv4eZDnHVL3lhKx
To: /content/paligemma_val.zip
100% 10.4M/10.4M [00:00<00:00, 47.5MB/s]


In [3]:
import torch
from transformers import PaliGemmaForConditionalGeneration, PaliGemmaProcessor, TrainingArguments, Trainer
from datasets import load_from_disk

In [4]:
def collate_fn(examples):
    texts = ["answer " + example["question"] for example in examples]
    labels= [example['answer'] for example in examples]
    images = [example["image"] for example in examples]
    tokens = processor(text=texts, images=images, suffix=labels,
                    return_tensors="pt", padding="longest")
    tokens = tokens.to(torch.bfloat16).to(device)
    return tokens

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = PaliGemmaForConditionalGeneration.from_pretrained("google/paligemma-3b-pt-224", torch_dtype=torch.bfloat16).to(device)
processor = PaliGemmaProcessor.from_pretrained("google/paligemma-3b-pt-224")



model-00001-of-00003.safetensors:  14%|#3        | 671M/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [8]:
import wandb
wandb.init(project="PaliGemmaOU", name="exp2")

In [10]:
args=TrainingArguments(
            num_train_epochs=5,
            remove_unused_columns=False,
            per_device_train_batch_size=4,
            gradient_accumulation_steps=1,
            warmup_steps=2,
            learning_rate=2e-5,
            weight_decay=1e-6,
            adam_beta2=0.999,
            logging_steps=100,
            optim="adamw_hf",
            save_strategy="epoch",
            push_to_hub=True,
            save_total_limit=1,
            bf16=True,
            dataloader_pin_memory=False,
            output_dir="v4",
            report_to="wandb",
            evaluation_strategy='epoch'
        )

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
for param in model.vision_tower.parameters():
    param.requires_grad = False

for param in model.multi_modal_projector.parameters():
    param.requires_grad = True

In [12]:
train_ds = load_from_disk('/content/paligemma_train/paligemma_train')
val_ds = load_from_disk('/content/paligemma_val/paligemma_val')

In [13]:
trainer = Trainer(
        model=model,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=collate_fn,
        args=args
        )
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as t

Epoch,Training Loss,Validation Loss
1,0.186700,0.120529
2,0.081200,0.137117
3,0.101200,0.124363
4,0.102700,0.131604
5,0.047500,0.130224


Streaming output truncated to the last 5000 lines.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how 

TrainOutput(global_step=12500, training_loss=0.0991575841140747, metrics={'train_runtime': 3723.8886, 'train_samples_per_second': 13.427, 'train_steps_per_second': 3.357, 'total_flos': 1.972917355478112e+17, 'train_loss': 0.0991575841140747, 'epoch': 5.0})

In [14]:
trainer.save_model()

In [15]:
trainer.state.log_history

[{'loss': 0.305,
  'grad_norm': 0.24609375,
  'learning_rate': 1.9843174907985278e-05,
  'epoch': 0.04,
  'step': 100},
 {'loss': 0.2498,
  'grad_norm': 4.5625,
  'learning_rate': 1.9683149303888623e-05,
  'epoch': 0.08,
  'step': 200},
 {'loss': 0.2043,
  'grad_norm': 0.169921875,
  'learning_rate': 1.952312369979197e-05,
  'epoch': 0.12,
  'step': 300},
 {'loss': 0.1718,
  'grad_norm': 2.046875,
  'learning_rate': 1.936309809569531e-05,
  'epoch': 0.16,
  'step': 400},
 {'loss': 0.1874,
  'grad_norm': 0.10986328125,
  'learning_rate': 1.9203072491598657e-05,
  'epoch': 0.2,
  'step': 500},
 {'loss': 0.221,
  'grad_norm': 0.0274658203125,
  'learning_rate': 1.9043046887502003e-05,
  'epoch': 0.24,
  'step': 600},
 {'loss': 0.1629,
  'grad_norm': 0.1455078125,
  'learning_rate': 1.8883021283405348e-05,
  'epoch': 0.28,
  'step': 700},
 {'loss': 0.1558,
  'grad_norm': 0.166015625,
  'learning_rate': 1.872299567930869e-05,
  'epoch': 0.32,
  'step': 800},
 {'loss': 0.1863,
  'grad_norm':

In [26]:
trainer.evaluate()

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
Y

{'eval_loss': 0.11926785111427307,
 'eval_runtime': 3.7204,
 'eval_samples_per_second': 53.757,
 'eval_steps_per_second': 6.72,
 'epoch': 5.0}

In [16]:
model_id = "woosterheert/v4"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16)

config.json:   0%|          | 0.00/993 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/862M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [17]:
results = []
for d in val_ds:
  texts = ["answer " + d["question"]]
  images = [d["image"]]
  tokens = processor(text=texts, images=images, return_tensors="pt", padding="longest")
  inputs = tokens.to(torch.bfloat16).to(device)
  model.to(device)
  with torch.no_grad():
    output = model.generate(**inputs, max_length=496)

  prediction = processor.decode(output[0], skip_special_tokens=True).split('\n')[1]
  results.append((prediction, d['answer']))

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
Y

In [18]:
score = [1 if x[0] == x[1] else 0 for x in results]

In [19]:
sum(score) / len(score)

0.955